from https://github.com/intive-DataScience/tbats/blob/master/examples/detailed_tbats.py

In [ ]:
import pandas as pd
import numpy as np
import datetime as dt

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# ! pip install tbats

In [ ]:
from tbats import BATS, TBATS

In [ ]:
if __name__ == '__main__':
    steps = 14
    np.random.seed(2342)
    t = np.array(range(0, 160))
    y = 5 * np.sin(t * 2 * np.pi / 7) + 5 * np.cos(t * 2 * np.pi / 30.25) + \
        ((t / 20) ** 1.5 + np.random.normal(size=160) * t / 50) + 10
    y = np.asarray(y)
    y_to_train = y[:(len(y) - steps)]
    y_to_predict = y[(len(y) - steps):]

In [ ]:
estimator = TBATS(
        seasonal_periods=[14.5, 30.25],
        use_arma_errors=None,  # shall try models with and without ARMA
        use_box_cox=False,  # will not use Box-Cox
        use_trend=None,  # will try models with trend and without it
        use_damped_trend=None,  # will try models with daming and without it
        show_warnings=False,  # will not be showing any warnings for chosen model
    )

In [ ]:
fitted_model = estimator.fit(y_to_train)

In [ ]:
# Warning messages from the model, if any
for warning in fitted_model.warnings:
    print(warning)

In [ ]:
print('Did the model fit?', fitted_model.is_fitted)  # Model may fail to fit in edge-case situations
print('AIC', fitted_model.aic)  # may be np.inf

Did the model fit? True
AIC 69727.21263550116


In [ ]:
# Lets check components used in the model
print('\n\nMODEL SUMMARY\n\n')
params = fitted_model.params
components = fitted_model.params.components



MODEL SUMMARY




In [ ]:
print('Smoothing parameter', params.alpha)

Smoothing parameter 0.14638137120476663


In [ ]:
print('Seasonal periods',
          components.seasonal_periods)  # TBATS may choose non-seasonal model even if you provide seasons
print('Harmonics amount for each season', components.seasonal_harmonics)
print('1st seasonal smoothing parameters', params.gamma_1())  # one value for each season
print('2nd seasonal smoothing parameters', params.gamma_2())  # one value for each season

Seasonal periods [  7.   365.25]
Harmonics amount for each season [3 1]
1st seasonal smoothing parameters [ 1.51479130e-08 -5.18968735e-09]
2nd seasonal smoothing parameters [2.17852355e-08 1.78665609e-08]


In [ ]:
print('Trend and damping', components.use_trend, components.use_damped_trend)
print('Trend', params.beta)
print('Damping', params.phi)

Trend and damping True True
Trend -0.02356179882178914
Damping 0.8000000001761762


In [ ]:
print('Use Box-Cox', components.use_box_cox)
print('Box-Cox lambda interval that was considered', components.box_cox_bounds)
print('Box-Cox lambda', params.box_cox_lambda)

Use Box-Cox False
Box-Cox lambda interval that was considered (0, 1)
Box-Cox lambda None


In [ ]:
print('ARMA residuals modelling', components.use_arma_errors)
print('ARMA(p, q)', components.p, components.q)
print('AR parameters', params.ar_coefs)
print('MA parameters', params.ma_coefs)

ARMA residuals modelling False
ARMA(p, q) 0 0
AR parameters []
MA parameters []


In [ ]:
print('Seed state', params.x0)

Seed state [ 2046.06248987   113.56905563  1784.12981697 -1343.42064111
   567.53486248 -2962.76571841   180.66241952   191.55113227
 -1129.8617667   3961.55985326]


In [ ]:
# Short model summary
print('\n\nSUMMARY FUNCTION\n\n')
print(fitted_model.summary())



SUMMARY FUNCTION


Use Box-Cox: False
Use trend: True
Use damped trend: True
Seasonal periods: [  7.   365.25]
Seasonal harmonics [3 1]
ARMA errors (p, q): (0, 0)
Smoothing (Alpha): 0.146381
Trend (Beta): -0.023562
Damping Parameter (Phi): 0.800000
Seasonal Parameters (Gamma): [ 1.51479130e-08  2.17852355e-08 -5.18968735e-09  1.78665609e-08]
AR coefficients []
MA coefficients []
Seed vector [ 2046.06248987   113.56905563  1784.12981697 -1343.42064111
   567.53486248 -2962.76571841   180.66241952   191.55113227
 -1129.8617667   3961.55985326]

AIC 69727.212636


In [ ]:
print('\n\nIN SAMPLE PREDICTIONS\n\n')
print('Original time series (5 first values)', fitted_model.y[:5])
print('Predictions (5 first values)', fitted_model.y_hat[:5])
print('Residuals (5 first values)', fitted_model.resid[:5])



IN SAMPLE PREDICTIONS


Original time series (5 first values) [0. 0. 0. 0. 0.]
Predictions (5 first values) [ 2015.30000604  -266.10781625  -867.26601342 -2482.85833473
   423.34441147]
Residuals (5 first values) [-2015.30000604   266.10781625   867.26601342  2482.85833473
  -423.34441147]


In [ ]:
y_forecasted = fitted_model.forecast(steps=steps)

In [ ]:
print('\n\nFORECAST\n\n')
print('Values', y_forecasted)
print('MAE', np.mean(np.abs(y_forecasted - y_to_predict)))



FORECAST


Values [27702.03372466 25997.93971865 24026.61276121 23387.58605805
 21645.97901207 24219.72989306 29907.2024981  28635.31516069
 26727.07355715 24590.74716738 23818.04523758 21967.82849886
 24453.02877764 30068.00520835 28736.47288432 26778.87537743
 24601.43296929 23794.2160302  21914.77467058 24374.99269748
 29968.3913041  28618.01570186 26643.77402495 24451.45866932
 23630.79815761 21739.06957942 24187.93877858 29770.75302719
 28410.41884528 26426.73385513 24225.40248009 23396.08329716
 21495.99786727 23936.76803445 29511.70628927 28143.69173982
 26152.50048713 23943.82014052 23107.29622349 21200.14024642
 23633.9663996  29202.0814316  27827.36023218 25829.57589697
 23614.41399433 22771.51877218 20858.10107139 23285.77489132
 28847.847166   27467.19326523 25463.58699377 23242.71480559
 22394.22196864 20475.32044443 22897.62542642 28454.44509446
 27068.65609837 25060.03354298 22834.26519111 21980.99761911
 20057.44408903 22475.22108106 28027.63804303 26637.57287006
 246

https://www.sktime.org/en/stable/api_reference/auto_generated/sktime.forecasting.tbats.TBATS.html#